In [13]:
import numpy as np
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score

In [9]:
df = pd.read_csv('../../data/bmi.csv', skiprows=3)

X_train, X_test, y_train, y_test = train_test_split(df[['height','weight']], df['label'],
                                                    test_size=0.3, random_state=1, stratify=df['label'])

In [10]:
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [12]:
model = LogisticRegression()
model.fit(X_train_scaled, y_train)
acc = model.score(X_test_scaled, y_test)
print('LogisticRegression의 Accuracy: {}'.format(acc))

LogisticRegression의 Accuracy: 0.9851666666666666


In [14]:
knn_classifier = KNeighborsClassifier(n_neighbors=3)
knn_classifier.fit(X_train_scaled, y_train)
acc = knn_classifier.score(X_test_scaled, y_test)
print('KNN의 Accuracy: {}'.format(acc))

KNN의 Accuracy: 0.9985


In [15]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [16]:
import pandas as pd
import numpy as np
from scipy import stats
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings('ignore')

In [18]:
df = pd.read_csv('../../data/ozone.csv')
df

,Ozone,Solar.R,Wind,Temp,Month,Day
0,41.0,190.0,7.4,67,5,1
1,36.0,118.0,8.0,72,5,2
2,12.0,149.0,12.6,74,5,3
3,18.0,313.0,11.5,62,5,4
4,NaN,NaN,14.3,56,5,5
...,...,...,...,...,...,...
148,30.0,193.0,6.9,70,9,26
149,NaN,145.0,13.2,77,9,27
150,14.0,191.0,14.3,75,9,28
151,18.0,131.0,8.0,76,9,29


In [19]:
x_data = df[['Solar.R', 'Wind', 'Temp']]
y_data = df['Ozone']

In [20]:
for col in x_data.columns:
    col_median = np.nanmedian(x_data[col])
    x_data[col].loc[x_data[col].isnull()] = col_median
    
x_data.isnull().sum()

Solar.R    0
Wind       0
Temp       0
dtype: int64

In [21]:
zscore_threshold = 2.0

for col in x_data.columns:
    outlier = x_data[col][np.abs(stats.zscore(x_data[col])) > zscore_threshold]
    col_mean = np.mean(x_data.loc[~x_data[col].isin(outlier), col])
    x_data.loc[x_data[col].isin(outlier), col] = col_mean

In [22]:
outlier = y_data[np.abs(stats.zscore(y_data)) > zscore_threshold]
col_mean = np.mean(~y_data.isin(outlier))
y_data[y_data.isin(outlier)] = col_mean

In [28]:
X_scaler = MinMaxScaler()
y_scaler = MinMaxScaler()

X_scaler.fit(x_data.values)
y_scaler.fit(y_data.values.reshape(-1, 1))

x_data_scaled = X_scaler.transform(x_data.values)
y_data_scaled = y_scaler.transform(y_data.values.reshape(-1, 1)).ravel()

train_x_data_scaled = x_data_scaled[~np.isnan(y_data_scaled)]
train_y_data_scaled = y_data_scaled[~np.isnan(y_data_scaled)]

In [30]:
knn_regression = KNeighborsRegressor(n_neighbors=2)
knn_regression.fit(train_x_data_scaled, train_y_data_scaled)

knn_predict = knn_regression.predict(x_data_scaled[np.isnan(y_data_scaled)])
y_data_scaled[np.isnan(y_data_scaled)] = knn_predict